In [184]:
import pandas as pd
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Bot')
import re
import warnings
warnings.filterwarnings("ignore")
import sqlite3
from tqdm import tqdm
from IPython.display import Audio, display
from sqlite3.dbapi2 import OperationalError, IntegrityError
from multiprocessing import Pool

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data-preparing

##Part0

In [ ]:
dt = pd.read_csv('movies1.csv', encoding='latin1', sep=';',\
                 names=['Title', 'Type', 'Year', 'Age Limit', 'Episodes', 'Time', 'Genre(s)', 'IMDB', 'Metascore', 'Overview', 'Directed by', 'Starring', 'Box Office', 'Url',\
                        0,1,2,3,4,5,6,7,8,9,10])
dt.shape

In [ ]:
dt.isna().sum()

NameError: ignored

In [ ]:
dt.drop([0,1,2,3,4,5,6,7,8,9], axis=1, inplace=True)

In [ ]:
dt[dt[0].isna()==False]

In [ ]:
#53453, 48889

In [ ]:
start=53453
dt[start:start+1]

In [ ]:
# start=
for i in range(start, start+1):
  dt['Year'][i] = '(2007)'
  dt['Directed by'][i] = 'Jeremy Saulnie'
  dt['Starring'][i] = 'Chris Sharp, Kate Porterfield, Tess Porterfield Lovell, Puff Snooty'
  dt['Box Office'][i] = np.nan
  dt['Url'][i] = '/title/tt0878695/?ref_=nv_sr_srsg_0'

NameError: ignored

In [ ]:
d = [['Hazbin Hotel', 'TV Series', '(2019– )', 'TV-MA', 2, '32 min', 'Animation, Comedy, Crime', 7.7, np.nan, \
      "In attempt to find a non-violent alternative for reducing Hell's overpopulation, the daughter of Lucifer opens a rehabilitation hotel that offers a group of misfit demons a chance at redemption.",\
      'Vivienne Medrano', 'Amir Hossein Amirpour, Jill Harris, Elsie Love, Michael Kovach', np.nan, '/title/tt7216636/?ref_=adv_li_tt'],\
     ['Wolfwalkers','Movie','(2020)','PG',np.nan,'103 min','Animation, Action, Adventure', 8, 87,\
      'A young apprentice hunter and her father journey to Ireland to help wipe out the last wolf pack. But everything changes when she befriends a free-spirited girl from a mysterious tribe rumored to transform into wolves by night.',\
      'Tomm Moore, Ross Stewart', 'Honor Kneafsey, Eva Whittaker, Sean Bean, Simon McBurney', np.nan, '/title/tt5198068/?ref_=adv_li_tt']
]
d = pd.DataFrame(columns=dt.columns.tolist(),data=d)
d

In [ ]:
dt = pd.concat([dt,d])
dt.reset_index(inplace=True, drop=True)
dt.tail()

In [ ]:
dt.to_csv('movies_corr.csv', index=True)

##part 1


###subpart1

In [ ]:
dt = pd.read_csv('movies_pr.csv')
dt.shape

(15569, 14)

In [ ]:
dt.drop_duplicates(inplace=True)
dt.reset_index(inplace=True, drop=True)
dt.shape

(21698, 14)

In [ ]:
start = 0
dt.Year.value_counts()[start:start+50]

Series([], Name: Year, dtype: int64)

In [ ]:
dt.to_csv('movies_pr.csv', index=False)

In [ ]:
for i in tqdm(range(len(dt))):
  if str(dt.Year[i]).__contains__('TV Special') == True:
    dt.Year[i] = str(dt.Year[i]).replace(' TV Special', '')
    dt.Type[i] = 'TV Special'
  elif str(dt.Year[i]).__contains__('TV Movie') == True:
    dt.Year[i] = str(dt.Year[i]).replace(' TV Movie', '')
    dt.Type[i] = 'TV Movie'
  elif str(dt.Year[i]).__contains__('TV Short') == True:
    dt.Year[i] = str(dt.Year[i]).replace(' TV Short', '')
    dt.Type[i] = 'TV Short'

100%|██████████| 21698/21698 [00:00<00:00, 31205.93it/s]


In [ ]:
dt.shape

(19553, 15)

In [ ]:
ind = dt[dt.IMDB.isna()].index
dt.drop(index=ind, inplace=True)
dt.reset_index(inplace=True)

In [ ]:
ind11 = dt[dt.Overview.str.contains('Plot|Post-production|Filming|See full summary')].index.tolist()
ind12 = dt[dt.IMDB.isna()].index.tolist()
ind13 = dt[(dt.Starring.isna())&(dt['Genre(s)'].str.contains('Animation')==False)].index.tolist()
ind14 = dt[(dt.Year.str.contains(r'Podcast Series|Video Game|Music Video|Video', na=False))].index.tolist()
inx = list(set(ind11 + ind12 + ind13 + ind14))
dt.drop(index=inx, inplace=True)
dt.drop_duplicates(inplace=True)
dt.shape

(19553, 14)

In [ ]:
# dt.drop('index', axis=1, inplace=True)
dt.drop(index=ind, inplace=True)
dt.reset_index(drop=True, inplace=True)
dt.shape

(15569, 14)

In [ ]:
ind = [5021, 6510, 6554, 5631, 6613, 750, 4831, 6624, 970, 5210,\
       1217, 5236, 6393, 13510, 6426, 6716, 6724, 5355, 9243, 10220,\
       13389, 3155, 5080, 4161, 9249, 6858, 6151, 4566, 5214, 5579,\
       5629, 6176, 9281, 5398, 9414, 6725, 6058, 15704, 13623, 15703,\
       9178, 13447, 13337, 13757]

In [ ]:
dt[dt.Overview.str.contains('When the brilliant but unorthodox scientist')]

,Title,Type,Year,Age Limit,Episodes,Time,Genre(s),IMDB,Metascore,Overview,Directed by,Starring,Box Office,Url
2025,Frankenstein,Movie,(1994),R,NaN,123 min,"Drama, Horror, Romance",6.3,49.0,When the brilliant but unorthodox scientist Dr...,Kenneth Branagh,"Robert De Niro, Kenneth Branagh, Helena Bonham...",$22.01M,/title/tt0109836/?ref_=adv_li_tt
14637,Frankenstein: The True Story,TV Movie,(1973),Not Rated,NaN,185 min,"Drama, Horror, Sci-Fi",7.2,NaN,When the brilliant but unorthodox scientist Dr...,Jack Smight,"James Mason, Leonard Whiting, David McCallum, ...",NaN,/title/tt0070074/?ref_=adv_li_tt


In [ ]:
dt[dt.duplicated(['Overview'])][7:]

,Title,Type,Year,Age Limit,Episodes,Time,Genre(s),IMDB,Metascore,Overview,Directed by,Starring,Box Office,Url
11664,Die Wannseekonferenz,TV Movie,(1984),NaN,NaN,87 min,"Drama, History",7.8,NaN,"At the Wannsee Conference on January 20, 1942,...",Heinz Schirk,"Dietrich Mattausch, Gerd B?ckmann, Friedrich G...",NaN,/title/tt0088377/?ref_=adv_li_tt
12624,The Missiles of October,TV Movie,(1974),NaN,NaN,150 min,"Drama, History, Thriller",8.1,NaN,"In October 1962, the Kennedy administration st...",Anthony Page,"William Devane, Ralph Bellamy, Howard Da Silva...",NaN,/title/tt0071847/?ref_=adv_li_tt
12855,Lady Oscar,Movie,(1979),NaN,NaN,124 min,"Drama, History, Romance",5.9,NaN,"The story of Lady Oscar, a female military com...",Jacques Demy,"Catriona MacColl, Barry Stokes, Patrick Allen,...",NaN,/title/tt0077827/?ref_=adv_li_tt
14637,Frankenstein: The True Story,TV Movie,(1973),Not Rated,NaN,185 min,"Drama, Horror, Sci-Fi",7.2,NaN,When the brilliant but unorthodox scientist Dr...,Jack Smight,"James Mason, Leonard Whiting, David McCallum, ...",NaN,/title/tt0070074/?ref_=adv_li_tt


###names

###Titles

In [ ]:
dt.to_csv('movies_pr.csv', index=False)

NameError: ignored

In [ ]:
dt = pd.read_csv('movies_pr.csv')
dt.shape

(15569, 15)

In [ ]:
ind = []
for i in range(len(dt)):
  if dt.Title[i].__contains__('?') and dt.Title[i][-1]!='?':
    ind.append(i)

In [ ]:
len(ind)

2

In [ ]:
start, step = 120, 3
dt.loc[ind][start:start+step]

,Title,Type,Year,Age Limit,Episodes,Time,Genre(s),IMDB,Metascore,Box Office,Overview,Directed by,Cast,Images,Url


In [ ]:
d = ['Alcarràs', 'Tár' , 'The Lørenskog Disappearance','Carnivàle', 'Léon: The Professional', 'Les Misérables', 'Sex and Lucía', 'Amélie',\
     'Salò, or the 120 Days of Sodom', '9½ Weeks', 'Les Misérables', 'Yali Çapkini', 'Händelser vid vatten', 'Sen Çal Kapimi', 'Y tu mamá también',\
     'Jamón, Jamón', 'Suite Française', 'The Double Life of Véronique', 'Les Misérables', 'Café Society','Beau-père', 'La Belle Époque', 'À Nos Amours',\
     'Együtt kezdtük','Ma mère', 'Another 9½ Weeks', 'Metástasis', 'Anaïs in Love', 'La Jetée', 'Günesin Kizlari','Tack för senast', 'Velvet Colección',\
     'Rurôni Kenshin: Sai shûshô - The Beginning', 'Pokémon', 'Pokémon: Detective Pikachu','Who Killed Santa? A Murderville Murder Mystery',\
     'The Naked Gun 2½: The Smell of Fear', 'Dejémonos de Vargas', 'Brüno', 'Çukur', 'Astrid et Raphaëlle', 'Çekiç ve Gül: Bir Behzat Ç. Hikayesi',\
     'Den som dræber', 'Gåsmamman', 'Pablo Escobar: El Patrón del Mal', 'Alarm für Cobra 11 - Die Autobahnpolizei', 'Içerde', 'Tunna blå linjen',\
     'Good Morning, Verônica', 'Jägarna', 'Les rivières pourpres', 'Forhøret', 'Försvunna människor', 'Æon Flux', 'Blodsbrødre', 'Belascoarán, PI',\
     'Aranyélet', 'Apollo 10½: A Space Age Childhood', 'Pokémon: The First Movie - Mewtwo Strikes Back', 'Astérix and Obélix: God Save Britannia',\
     'Pokémon the Movie 2000', 'Der Räuber Hotzenplotz', 'The Extraordinary Adventures of Adèle Blanc-Sec', 'Häxan', 'Pokémon Master Journeys',\
     "À l'aventure", 'Skyggen i mit øje', 'Glória', 'Uyanis: Büyük Selcuklu', 'Alparslan: Büyük Selçuklu', 'The Baby of Mâcon',\
     'Yesilçam: Bir Sinema Hayvani', 'Caché (Hidden)', 'Walk Invisible: The Brontë Sisters', 'Muhtesem Yüzyil: Kösem', 'Évolution', 'Napoléon',\
     'Das Weiße Haus am Rhein', 'Junji Itô: Korekushon', 'Hernán', 'Payitaht Abdülhamid', 'Désirée', 'D.B. Cooper: Where Are You?!', 'Águila Roja',\
     'Les Misérables', 'Napszállta', 'Zlociny Velké Prahy', 'Les Misérables: The Staged Concert', 'María Félix: La Doña', 'Angélique',\
     'The Brontë Sisters', 'Die Geschichte der Menschheit - leicht gekürzt', 'Angélique', 'O Hóspede Americano', 'Hôtel Terminus', 'Zátopek',\
     'Tarkan: Gümüs Eyer', 'Lou Andreas-Salomé, The Audacity to be Free', 'Ministério do Tempo', 'Les Misérables', 'Kautokeino-opprøret',\
     'Buñuel in the Labyrinth of the Turtles', 'Merveilleuse Angélique', 'La Línea Invisible', 'Mio fratello è figlio unico', 'Hem till Midgård',\
     'Verónica', 'Bijitâ Q', 'Arthur, malédiction', 'Cube²: Hypercube', 'Jisatsu sâkuru', 'Malasaña 32', 'Nausicaä of the Valley of the Wind',\
     'García!', 'Inspector Sun y la maldición de la viuda negra', 'Ningen Fushin no Bôkensha-tachi ga Sekai wo Sukû Yô Desu',\
     'Pokémon the Movie: I Choose You!', 'Pokémon 3 the Movie: Spell of the Unown', 'Pokémon: Mewtwo Strikes Back - Evolution',\
     'Pokémon the Movie: Secrets of the Jungle', 'Ôyukiumi no Kaina', 'Isekai nonbiri nôka']
dl = {i:j for i,j in zip(ind, d)}

In [ ]:
list(dl.items())[-10:]

[(5494, 'Alcarràs'), (11730, 'Tár')]

In [ ]:
for k,v in dl.items():
  dt.Title[k] = v

In [ ]:
dt.to_csv('movies_pr.csv', index=False)

###Overviews

In [ ]:
dt = pd.read_csv('movies_pr.csv')
dt.shape

(15569, 14)

In [ ]:
ind, k = [], []
for i in tqdm(range(len(dt))):
  if dt.Overview[i].__contains__('?') and dt.Overview[i][-1]!='?':
    ind.append(i)
    words = dt.Overview[i].split()
    for word in words:
      if word.__contains__('?') and word not in k:
        k.append(word)


100%|██████████| 15569/15569 [00:00<00:00, 99780.15it/s]


In [ ]:
len(ind), len(k)

(252, 230)

In [ ]:
dt.Title[26] = '8½'
dt.Title[2389] = '90 Day Fiancé'
dt.Title[4616] = 'Ófærð'
dt.Title[6117] = 'Kona fer í stríð'
dt.Title[7255] = 'Le Samouraï'
dt.Title[8153] = 'The Protégé'
dt.Title[10491] = 'Project Mc²'
dt.Title[10885] = 'Liberté'
dt.Title[11408] = 'Charité'
dt.Title[11556] = 'Marketa Lazarová'
dt.Title[12163] = 'Alien³'
dt.Title[12843] = 'Be Natural: The Untold Story of Alice Guy-Blaché'
dt.Title[13037] = 'Salomé'
dt.Title[13528] = '[Rec]²'
dt.Title[14352] = 'A White Dress for Marialé'
dt.Title[15465] = 'Ranma ½'
dt.Overview[14705] = 'War-hungry teddy bears journey from bootcamp to the psychedelic terrors of the Magic Forest in this darkly beautiful horror animation.'

In [ ]:
dt.Overview[dt.Overview.str.contains('unicorn', )]

5514     An injured hitman befriends his kidnapped daug...
6093     After learning that her friends, as well as he...
8964     A beautiful unicorn sets out to learn if she t...
10304    A live-action girl named Mia is plunged into a...
14705    This story tracks two brother teddy bear soldi...
15479    The fan-favorite cat-unicorn hybrid from The L...
Name: Overview, dtype: object

In [ ]:
start, step = 14705, 1
dt[start:start+step]

,Title,Type,Year,Age Limit,Episodes,Time,Genre(s),IMDB,Metascore,Overview,Directed by,Starring,Box Office,Url
14705,Unicorn Wars,Movie,(2022),18+,NaN,80 min,"Animation, Action, Comedy",6.9,NaN,War-hungry teddy bears journey from bootcamp t...,Alberto V?zquez,"Jon Goiri, Ram?n Barea, Maribel Legarreta, Itx...",NaN,/title/tt10483152/?ref_=adv_li_tt


In [ ]:
start, step = 230, 5
k[start:start+step]

[]

In [ ]:
start, step = 250, 5
dt.loc[ind, ['Title', 'Year', 'Overview']][start:start+step].values

array([['Speed Racer', '(1967–1968)',
        "Teenager G? Mifune aspires to be the world's best race-car champion with the help of his friends, family and his father's high-tech race-car, the Mach 5."],
       ['Maria Clara and Ibarra', '(2022– )',
        "Klay, a Gen Z nursing student, is transported into Jos? Rizal's novels after reading Noli Me T?ngere. There, she meets Maria Clara, Crisostomo Ibarra, and other characters."]],
      dtype=object)

In [ ]:
d = {'prot?g?':'protégé', 'T?r':'Tár', 'fianc?':'fiancé', 'Bogot?': 'Bogotá', 'Sof?a':'Sofía', '?milie':'Émilie', '?lex':'Álex', 'na?ve':'naïve',\
     'L?o':'Léo', "R?mi":"Rémi", 'Mal?na':'Malèna', 'Ad?le':'Adèle', "d'?tat":"d'état",'Op?ra':'Opéra', 'proteg?':'protégé', 'Merl?':'Merlí',\
     'J?gerst?tter':'Jägerstätter', ' caf?':' café', 'd?butante':'débutante', 'client?le':'clientèle', 'divorc?':'divorcé', 'H?l?ne':'Hélène',\
     '?dith':'Édith', 'Ars?ne':'Arsène', "d\'?garement":"d'égarement", 'Chlo?':'Chloé', 'S?o Paulo':'São Paulo', 'fa?ade':'façade', 'attach?':'attaché',\
     'S?nchez':'Sánchez','Mu?ez':'Muñez', 'Atticus P?nd':'Atticus Pünd','?ber rich':'über rich', 'Jacob St?rr':'Jacob Störr','the Le?ncio':'the Leôncio',\
     'the Marru?':'the Marruá', 'Am?lia':'Amélia', 'm?nage ? trois':'ménage à trois', 'M?tley Cr?e':'Mötley Crüe','9? Weeks':'9½ Weeks', 'M?ria':'Mária',\
     'Jos?':'José', 'Mart?n':'Martín', 'Ana?s':'Anaïs', 'Caf?':'Café', 'Fr?d?ric':'Frédéric', 'Ra?l':'Raúl', 'Ragnar?k': 'Ragnarök',\
     'imp Blitz?':'imp Blitzø', 'Pok?mon': 'Pokémon', '\x92':"'", '?50k':'£50k', 'Egbert Sous?':'Egbert Sousé', 'Son Gok?':'Son Gokû', ' Ren?':' René',\
     'Diakit?':'Diakité', 'Ast?rix':'Astérix', 'Ob?lix':'Obélix', 'Num?robis':'Numérobis', 'Ram?n': 'Ramón', 'D?sseldorf':'Düsseldorf', 'Br?no':'Brüno',\
     'Medell?n':'Medellín','Saga Nor?n':'Saga Norén', 'Ko?ovali':'Koçovali', '?eki? ve G?l':'Çekiç ve Gül', 'Rapha?l':'Raphaël', '?mer U?ar':'Ömer Uçar',\
     'Mill?n':'Millán','Malm?':'Malmö', 'doppelg?nger':'doppelgänger','Th?r?se':'Thérèse','Jens M?ller':'Jens Møller', 'Bj?rn Andresen':'Björn Andresen',\
     'Bj?rn':'Bjørn', '?on Flux':'Æon Flux', 'soup?on':'soupçon', 'Neuk?lln':'Neukölln', 'Carl M?rck':'Carl Mørck', 'Bouvet?ya':'Bouvetøya',\
     'Faberg?':'Fabergé', "Tot? u Curtu":'Totò u Curtu', 'expos?':'exposé', 'Montr?al':'Montréal', 'H?ctor':'Héctor', 'Belascoar?n':'Belascoarán',\
     '?ngela':'Ángela', 'Jordn?ra':'Jordnära', 'Padm?':'Padmé', 'Khlo?':'Khloé', 'In?s':'Inés', 'clich?':'cliché', 'Z? Carioca':'Zé Carioca',\
     'Kung F?hrer':'Kung Führer', 'Bu?uel':'Buñuel', 'Dal?':'Dalí', 'Lou?s':'Louÿs', 'R?gnier':'Régnier', '?rp?d':'Árpád', 'Gl?ria':'Glória',\
     'Jo?o':'João','Per?n':'Perón', 'G?k Han':'Gök Han', 'D?az':'Díaz', 'Sebasti?o':'Sebastião', 'Bront?':'Brontë','?1 million':'£1 million',\
     'H?kon':'Håkon', 'Sobib?r':'Sobibór', 'Charit?':'Charité', 'Mar?a':'María', 'Due?as':'Dueñas', 'C?vennes':'Cévennes','Cond?':'Condé',\
     'Hern?n':'Hernán', 'Cort?s':'Cortés', 'M?xico':'México', 'Sm?land':'Småland', 'S?mi':'Sámi', 'Asunci?n':'Asunción', 'Ern?':'Ernõ', 'F?in':'Féin',\
     '?guila':'Águila', 'Lo?e':'Loïe', 'Berg?res':'Bergères', 'Lumi?re':'Lumière', 'P?ricourt':'Péricourt', 'C?line':'Céline', 'G?ring':'Göring',\
     'Assembl?e':'Assemblée', 'F?lix':'Félix', 'Ang?lique':'Angélique', 'derg?h':'dergâh', 'C?ndido':'Cândido', 'Kitzb?hel':'Kitzbühel',\
     'Superman?s':"Superman's", '(n?e ':'(née ', 'Z?topek':'Zátopek', 'Salom?':'Salomé', 'Fran?ois':'François', 'Guy-Blach?':'Guy-Blaché',\
     'Hor?kov?':'Horáková', 'Ant?nio':'António', '?lvares':'Álvares', 'Ic?ar Bolla?n':' Icíar Bollaín', 'M?rbult':'Mörbult', 'Midg?rd':'Midgård',\
     'H?j?':'Hōjō', '(K?ki)':'(Kôki)', 'N?dasdy':'Nádasdy', 'Ib??ez':'Ibáñez', 'Nausica?':'Nausicaä', 'Guant?namo':'Guantánamo', 'h?mophage':'hæmophage',\
     'Sl?born':'løborn', 'Ch?nin':'Chûnin', 'Bat?':'Batô', 'C?lestine':'Célestine','G? Mifune':'Gô Mifune', 'T?ngere':'Tángere' }

In [ ]:
for i in tqdm(range(len(dt))):
  dt.Overview[i] = dt.Overview[i].replace("\\'", "'")

100%|██████████| 15569/15569 [00:07<00:00, 2134.14it/s]


In [ ]:
start, step = 240, 10
dt.loc[ind, ['Title', 'Year', 'Overview']][start:start+step].values
print(dt.loc[(dt.Overview.str[-1]!='?') &(dt.Overview.str.contains('\?')),['Title', 'Overview']].values.shape)
dt.loc[(dt.Overview.str[-1]!='?') &(dt.Overview.str.contains('\?')),['Title', 'Overview']].values

(36, 2)


array([['Redeeming Love',
        'Sold into prostitution as a child, Angel knows nothing but betrayal. Can her heart ever be mended? Based upon the novel by Francine Rivers.'],
       ['Mr. Nobody',
        "A boy stands on a station platform as a train is about to leave. Should he go with his mother or stay with his father? Infinite possibilities arise from this decision. As long as he doesn't choose, anything is possible."],
       ['Patrick the Pug',
        'Sarah is a young woman whose life is in a bit of a mess. The last thing she needs is someone else to look after. Yet, like it or not, her Grandmother has bequeathed her a very spoiled pug - Patrick. Surely she must have had her reasons?.'],
       ['Love, Rosie',
        "Rosie and Alex have been best friends since they were 5, so they couldn't possibly be right for one another...or could they? When it comes to love, life and making the right choices, these two are their own worst enemies."],
       ['Slumdog Millionaire',
   

In [ ]:
dt.to_csv('movies_pr.csv', index=False)

###actor names

In [ ]:
dt = pd.read_csv('movies_pr.csv')
dt.shape

(15569, 14)

In [ ]:
ind, k = [], []
for i in tqdm(range(len(dt))):
  if str(dt.Starring[i]).__contains__('?'):
    ind.append(i)
    words = dt.Starring[i].split(', ')
    for word in words:
      if word.__contains__('?') and word not in k:
        k.append(word)
  if str(dt['Directed by']).__contains__('?'):
    ind.append(i)
    words = dt['Directed by'][i].split(', ')
    for word in words:
      if word.__contains__('?') and word not in k:
        k.append(word)
ind = list(set(ind))

100%|██████████| 15569/15569 [00:12<00:00, 1207.55it/s]


In [ ]:
s = []
for i in tqdm(range(len(dt))):
  words = str(dt.Starring[i]).split(', ')
  word = str(dt['Directed by'][i]).split(', ')
  s = s + words + word
s = list(set(s))

100%|██████████| 15569/15569 [00:10<00:00, 1483.09it/s]


In [ ]:
s = sorted(s, key=lambda x: x.split()[-1])
stars = pd.DataFrame({'name':s})

In [ ]:
len(ind), len(k)

(1782, 1404)

In [ ]:
stars[stars.name.str.contains('str\?m')]

,name
7841,Alexandra Dahlstr?m
10863,Lasse Fogelstr?m
13246,Mikael H?fstr?m
13247,Dan H?fstr?m
13476,Lasse Hallstr?m
14921,Pelle Holmstr?m
16880,Matilda K?llstr?m
19841,Kristina Lindstr?m
28841,Storm Acheche Sahlstr?m
30639,Victor Sj?str?m


In [ ]:
start, step = 50, 5
dt.loc[ind, ['Title', 'Year', 'Directed by', 'Starring']][start:start+step].values

array([['The Magnificent Century', '(2011–2014)', nan,
        'Halit Ergen?, Nur Fettahoglu, Selim Bayraktar, Selen ?zt?rk'],
       ['Buried', '(2010)', 'Rodrigo Cort?s',
        'Ryan Reynolds, Jos? Luis Garc?a-P?rez, Robert Paterson, Stephen Tobolowsky'],
       ['Cop Out', '(2010)', 'Kevin Smith',
        'Bruce Willis, Tracy Morgan, Juan Carlos Hern?ndez, Cory Fernandez'],
       ['Annie', '(2014)', 'Will Gluck',
        'Quvenzhan? Wallis, Cameron Diaz, Jamie Foxx, Rose Byrne'],
       ['Becoming Elizabeth', '(2022)', 'Anya Reiss',
        'Alicia von Rittberg, John Heffernan, Oliver Zetterstr?m, Jamie Parker']],
      dtype=object)

In [ ]:
start, step = 28, 4
k[start:start+step]

['Ulrich M?he', 'B?r?nice Bejo', 'Halit Ergen?', 'Selen ?zt?rk']

In [ ]:
n = {'V?ra':'Véra', 'Y?ki':'Yûki', 'Kensh?':'Kenshô', 'Skarsg?rd':'Skarsgård', 'Aim?e':'Aimée', 'Andr?':'André', 'Zo?':'Zoë', 'Trevi?o':'Treviño',\
     'Mikko P?ll?':'Mikko Pöllä','Markku P?l?nen':'Markku Pölönen', 'Gy?rgy P?lfi':'György Pálfi', 'P?l':'Pål', 'No?mie':'Noémie', 'Pen?lope':'Penélope',\
     'Ram?rez':'Ramírez', 'Garc?a':'García', 'J?rgensen':'Jørgensen', 'Nils J?rgen':'Nils Jørgen', 'J?rgen Liik':'Jörgen Liik', 'Guzm?n':'Guzmán',\
     'J?rgen Thorsson':'Jörgen Thorsson','J?rgen':'Jürgen', 'Tch?ky':'Tchéky', 'K?ji':'Kôji', 'R?gis':'Régis', 'Micha?l':'Michaël','Fran?ois':'François',\
     'Go?chy':'Goûchy', 'Bj?rn Floberg':'Bjørn Floberg', 'Bj?rn Myrene':'Bjørn Myrene', 'Bj?rnar Bruun':'Bjørn Bruun', 'Anne Bj?rnstad':'Anne Bjørnstad',\
     'Bj?rn':'Björn', 'N?lisse':'Nélisse', 'Chlo? Zhao':'Chloé Zhao', 'Chlo?':'Chloë', 'Tetsur?':'Tetsurô', 'St?en':'Støen', 'Kit?':'Kitô',\
     'Gonz?lez':'González', 'M?nguez':'Mínguez', 'Gj?lme':'Gjølme', 'Kr?likowski':'Królikowski', 'Pratic?':'Praticò', 'H?seyin':'Hüseyin', 'Pe?a':'Peña',\
     'Gio?':'Gioè', 'Morz?':'Morzé', 'Ry?s':'Ryûs', 'Ry?ichi':'Ryûichi', 'Ry?z?':'Ryûzô', 'Ry?nosuke':'Ryûnosuke', 'Daisuke Ryû':'Daisuke Ryû',\
     'Ry?':'Ryô', 'Sh?ta Sometani':'Shôta Sometani', 'Mar?al For?s', 'Mar?a':'María', 'Ken?fic':'Kenéfic', 'C?cile':'Cécile','G?rard':'Gérard',\
     'Casta?eda':'Castañeda', 'G?tz':'Götz', 'Gu?r?n ?r Eyfj?r?':'Guðrún Ýr EyfjörðÍris', '?ris':'Íris', 'M?he':'Mühe', 'Moll?':'Mollà',\
     'Sigurj?n':'Sigurjón','Korm?kur':'Kormákur', 'B?r?nice':'Bérénice', 'Ergen?':'Ergenç', '?zt?rk':'Öztürk', 'Berl?and':'Berléand', 'H?bner':'Hübner', \
     'Sof?a':'Sofía','P?rez-':'Pérez', 'Fogelstr?m':'Fogelstrøm', 'H?fstr?m':'Håfström', 'K?llstr?m':'Källström', 'Sj?str?m':'Sjöström',\
     'Sahlstr?m':'Sahlstrøm', 'str?m':'ström', 'Quvenzhan?':'Quvenzhané', 'Hern?ndez':'Hernández', 


         }

### full cast

In [ ]:
dt = pd.read_csv('movies_pr.csv')
dt.head(1)
dt.shape

(15569, 14)

In [ ]:
names = ['Url', 'Directed', 'Cast']
df = pd.concat([pd.read_csv('actors.csv', names=names),\
                pd.read_csv('actors1.csv', names=names),\
                pd.read_csv('actors2.csv', names=names)])
df.reset_index(drop=True, inplace=True)
df.shape

(15559, 3)

In [ ]:
dt['Directed'] = [0]*len(dt)
dt['Cast'] = [0]*len(dt)

In [ ]:
for i in tqdm(range(len(dt))):
    if dt.Url[i] != df.Url[i]:
        print(i)
        break

 77%|███████▋  | 11986/15569 [00:00<00:00, 52869.56it/s]

11986


In [ ]:
dt.Directed[:11986] = df.Directed[:11986]
dt.Cast[:11986] = df.Cast[:11986]   
dt.Directed[11996:] = df.Directed[11986:]
dt.Cast[11996:] = df.Cast[11986:]        

In [ ]:
dt.Type[11986], dt.Episodes[11986] = 'TV Mini Series', 3
dt.Directed[11986], dt.Cast[11986] = np.nan, np.nan
dt.Directed[11987] = 'Henry Hathaway'
dt.Cast[11987] = 'Tyrone Power, Orson Welles, Cécile Aubry, Jack Hawkins, Michael Rennie, Finlay Currie, Herbert Lom, Mary Clare, Robert Blake, Alfonso Bedoya, Gibb McLaughlin, James Robertson Justice, Henry Oscar, Laurence Harvey, Itto Bent Lahcen, Rufus Cruickshank, Peter Drury, Valéry Inkijinoff'
dt.Directed[11988] = 'Alice Winocour'
dt.Cast[11988] = 'Vincent Lindon, Stéphan Wojtowicz, Soko, Chiara Mastroianni, Olivier Rabourdin, Roxane Duran, Lise Lamétrie, Ange Ruzé, Grégory Gadebois, Valentine Herrenschmidt, Aliénor de Mézamat, Audrey Bonnet, Jeanne Cohendy, Jean-Claude Baudracco, Julie Ravix, Victoire Gonin-Labat, Jacob Vouters, Damien Bonnard'
dt.Directed[11989] = 'Patrick McGoohan'
dt.Cast[11989] = 'Patrick McGoohan, Angelo Muscat, Peter Swanwick, Peter Brace, Leo McKern, Christopher Benjamin, Michael Miller, Alexis Kanner, Bill Cummings, Frank Maher, Patrick Cargill, Colin Gordon, Kenneth Griffith, Georgina Cookson, Harold Berens, John Cazabon, Bee Duffell, John Maxim'
dt.Directed[11990] = np.nan
dt.Cast[11990] = 'Mark Cousins, Juan Diego Botto, Aleksandr Sokurov, Norman Lloyd, Lars von Trier, Paul Schrader, Haskell Wexler, Woo-Ping Yuen, Robert Towne, Samira Makhmalbaf, Jean-Michel Frodon, Stanley Donen, Sharmila Tagore, Mani Kaul, Youssef Chahine, Kyôko Kagawa, Donald Richie, Gaston Kaboré'
dt.Directed[11991] = 'George Schaefer'
dt.Cast[11991] = 'Anthony Hopkins, Richard Jordan, Cliff Gorman, James Naughton, Michael Lonsdale, Martin Jarvis, Michael Kitchen, Andrew Ray, Piper Laurie, Susan Blakely, Robert Austin, Geoffrey Bateman, Graham Bishop, Kevin Bishop, Nathalie Boulmer, Yves Brainville, Jane Carr, Georges Corraface'
dt.Directed[11992] = 'Raoul Peck'
dt.Cast[11992] = 'August Diehl, Stefan Konarske, Vicky Krieps, Olivier Gourmet, Hannah Steele, Alexander Scheer, Hans-Uwe Bauer, Michael Brandner, Ivan Franek, Peter Benedict, Niels-Bruno Schmidt, Marie Meinzenbach, Wiebke Adam, Aran Bert, Ronald Beurms, Cédric Boulanger, Ulrich Brandhoff, Sarah Christoyannis'
dt.Directed[11993] = np.nan
dt.Cast[11993] = 'Christa Théret, Jannis Niewöhner, Alix Poisson, Jean-Hugues Anglade, Miriam Fussenegger, Stefan Pohl, Nicolas Wanczycki, Sylvie Testud, André Penvern, Raphaël Lenglet, Johannes Krisch, Fritz Karl, Harald Windisch, Thierry Piétra, Lili Epply, Max Baissette de Malglaive, Sebastian Blomberg, Christoph Luser'
dt.Directed[11994] = 'James Ivory'
dt.Cast[11994] = 'Nick Nolte, Greta Scacchi, Gwyneth Paltrow, Estelle Eonnet, Thandiwe Newton, Seth Gilliam, Todd Boyce, Nigel Whitmey, Nicolas Silberg, Catherine Samie, Lionel Robert, Stanislas Carré de Malberg, Jean Rupert, Yvette Petit, Paolo Mantini, Frédéric van den Driessche, Humbert Balsan, Nichel Rois'
dt.Directed[11995] = 'Stéphanie Di Giusto'
dt.Cast[11995] = 'Soko, Gaspard Ulliel, Mélanie Thierry, Lily-Rose Depp, François Damiens, Louis-Do de Lencquesaing, Amanda Plummer, Denis Ménochet, Charlie Morgan, Tamzin Merchant, William Houston, Bert Haelvoet, Camille Rutherford, Laurent Manzoni, Matilda Kime, Christian Erickson, Nicolas Helpiquet, Daniel Kramer'



In [ ]:
dt.drop(['Directed by', 'Starring'], axis=1, inplace=True)
dt.head(1)

,Title,Type,Year,Age Limit,Episodes,Time,Genre(s),IMDB,Metascore,Overview,Box Office,Url,Directed,Cast
0,Conversations with Friends,TV Series,(2022),TV-MA,12.0,335 min,"Drama, Romance",6.7,NaN,"Two Dublin college students, Frances and Bobbi...",NaN,/title/tt11997646/?ref_=adv_li_tt,NaN,"Alison Oliver, Sasha Lane, Joe Alwyn, Jemima K..."


In [ ]:
dt[11995:12000]

,Title,Type,Year,Age Limit,Episodes,Time,Genre(s),IMDB,Metascore,Overview,Box Office,Url,Directed,Cast
11995,The Dancer,Movie,(2016),Not Rated,NaN,108 min,"Biography, Drama, History",6.5,44.0,Loïe Fuller was the toast of the Folies Bergèr...,NaN,/title/tt4632440/?ref_=adv_li_tt,Stéphanie Di Giusto,"Soko, Gaspard Ulliel, Mélanie Thierry, Lily-Ro..."
11996,Neruda,Movie,(2016),R,NaN,107 min,"Biography, Crime, Drama",6.8,82.0,An inspector hunts down Nobel Prize-winning Ch...,$0.94M,/title/tt4698584/?ref_=adv_li_tt,Pablo Larraín,"Gael García Bernal, Luis Gnecco, Mercedes Morá..."
11997,Transformers: Revenge of the Fallen,Movie,(2009),PG-13,NaN,149 min,"Action, Adventure, Sci-Fi",5.9,35.0,Sam Witwicky leaves the Autobots behind for a ...,$402.11M,/title/tt1055369/?ref_=adv_li_tt,Michael Bay,"Shia LaBeouf, Megan Fox, Josh Duhamel, Tyrese ..."
11998,The Lion in Winter,TV Movie,(2003),NaN,NaN,167 min,"Drama, History, Romance",7.0,NaN,King Henry II meets with Eleanor of Aquitaine ...,NaN,/title/tt0319754/?ref_=adv_li_tt,Andrey Konchalovskiy,"Glenn Close, Andrew Howard, Antal Konrád, John..."
11999,Team Foxcatcher,Movie,(2016),TV-14,NaN,90 min,"Documentary, Biography, Crime",7.3,NaN,Documentary filmmaker Jon Greenhalgh examines ...,NaN,/title/tt5598216/?ref_=adv_li_tt,Jon Greenhalgh,"John du Pont, Dave Schultz, Mike Gostigian, Ra..."


In [ ]:
ind = []
for i in tqdm(range(len(dt))):
    if str(dt.Directed[i]) != 'nan':
        if '(' in dt.Directed[i]:
            dir = dt.Directed[i].split(', ')
            dir = [t.split('(')[0] for t in dir if '(' in t]
            a = ', '.join(dir)
            dt.Directed[i] = a

100%|██████████| 15569/15569 [00:00<00:00, 19830.20it/s]


In [ ]:
dt[dt.Type.str.contains('Series')][10:15]

,Title,Type,Year,Age Limit,Episodes,Time,Genre(s),IMDB,Metascore,Overview,Box Office,Url,Directed,Cast
34,Skam,TV Series,(2015–2017),Not Rated,43.0,30 min,"Drama, Romance",8.6,NaN,The story of young teenagers and pupils on Har...,NaN,/title/tt5288312/?ref_=adv_li_tt,Julie Andem,"Josefine Frida Pettersen, Iman Meskini, Ulrikk..."
46,Switched at Birth,TV Series,(2011–2017),TV-14,105.0,60 min,"Drama, Family, Romance",7.7,NaN,Two teenage girls' lives are turned upside dow...,NaN,/title/tt1758772/?ref_=adv_li_tt,Lizzy Weiss,"Katie Leclerc, Vanessa Marano, Constance Marie..."
50,A Teacher,TV Mini Series,(2020),TV-MA,10.0,248 min,Drama,6.9,NaN,A Teacher explores the complexities and conseq...,NaN,/title/tt10680614/?ref_=adv_li_tt,Hannah Fidell,"Kate Mara, Nick Robinson, Ashley Zukerman, Rya..."
51,Wilder,TV Series,(2017– ),NaN,24.0,NaN,"Crime, Drama, Mystery",7.4,NaN,A detective returns to his home town in the Sw...,NaN,/title/tt6276768/?ref_=adv_li_tt,Béla Batthyany,"Sarah Spale, Marcus Signer, Andreas Matti, Pet..."
52,Chapelwaite,TV Series,(2021– ),TV-MA,10.0,516 min,"Drama, Horror, Mystery",7.2,NaN,"In the 1850s, Captain Charles Boone relocates ...",NaN,/title/tt11525188/?ref_=adv_li_tt,"Jason Filardi, Peter Filardi","Adrien Brody, Emily Hampshire, Jennifer Ens, S..."


In [ ]:
dt.to_csv('movies_pr.csv', index=False)

##Images

In [ ]:
y = []
for i in tqdm(range(len(dt))):
    t = dt.Title[i]
    t = re.sub("[A-z0-9?\/\\';:'\|* ]", '', t)
    for j in t:
        if j not in y:
            y.append(j)

100%|██████████| 15569/15569 [00:00<00:00, 54205.66it/s]


In [ ]:
dt['Images'] = [0]*len(dt)

In [ ]:
k = []
for i in tqdm(range(len(dt))):
    if str(dt.Year[i]) != 'nan':
        y = '_' + re.sub('[\(\)]', '', str(dt.Year[i])).strip()
    else: y = ''
    im = re.sub("[?\/\\';:\*,!.]", '', dt.Title[i]).replace('"', '')
    im = im.replace('½', '1_2').replace('²', '2').replace('·', '-').replace('³', '3')
    im = '_'.join(im.lower().split(' ')) + y +'.jpg'
    if im in k:
        im = im[:-4] + '_1.jpg'
        dt['Images'][i] = im
        k.append(im)
    else:
        dt['Images'][i] = im
        k.append(im)

100%|██████████| 15569/15569 [00:10<00:00, 1495.21it/s]


In [ ]:
dt[dt.Title=='The Cursed']

,Title,Type,Year,Age Limit,Episodes,Time,Genre(s),IMDB,Metascore,Overview,Box Office,Url,Directed,Cast,Images
9477,The Cursed,Movie,(2021),R,NaN,111 min,"Fantasy, Horror, Mystery",6.3,62.0,"In rural 19th-century France, a mysterious, po...",NaN,/title/tt9907782/?ref_=adv_li_tt,Sean Ellis,"Boyd Holbrook, Kelly Reilly, Alistair Petrie, ...",the_cursed_2021.jpg
14697,The Cursed,Movie,(2021),NaN,NaN,110 min,Horror,6.0,NaN,A mysterious serial murder cases take place by...,NaN,/title/tt17640714/?ref_=adv_li_tt,Yong-wan Kim,"Uhm Ji-won, Jung Ji-so, Kim In-kwon, Seol Lee,...",the_cursed_2021_1.jpg


In [ ]:
dt.to_csv('movies_pr.csv', index=False)

In [ ]:
dt[dt.duplicated('Images')]

,Title,Type,Year,Age Limit,Episodes,Time,Genre(s),IMDB,Metascore,Overview,Box Office,Url,Directed,Cast,Images


#Tables preparing

##movies

In [ ]:
films = dt.copy()
films.drop(['Genre(s)', 'Cast', 'Directed', 'Url'], axis=1, inplace=True)

In [ ]:
n = [str(i) for i in range(10)]
ind = []
flag = False
for i in tqdm(range(len(films))):
    line = films.Title[i]
    line = re.sub(r"[\W]", '', line)
    line = re.sub(r"[ÀÁÂÄÅÆ]", 'A', line).replace('Ç', 'C').replace('Þ', 'Th')
    line = re.sub(r"[ÈÉÊ]", 'E', line).replace('Í', 'I').replace('ß', 'Ss')
    line = re.sub(r"[ÒÓÔÕÖØ]", 'O', line).replace('Î', 'I')
    line = re.sub(r"[ÚÜ]", 'U', line).replace('Ý', 'Y')
    l_part = re.sub(r'(a|à|á|â|ã|ä|å|æ|e|è|é|ê|ë|i|ì|í|î|ï|o|ò|ó|ô|õ|ö|ő|ø|u|ù|ú|ū|û|ü|y|ý|ÿ)', '', line).lower()
    l_part = l_part.replace('þ', 'th').replace('ñ', 'n').replace('ð', 'd').replace('½', '12').replace('ç', 'c')
    d_part = ''.join(np.random.choice(n, 5))
    index = l_part + d_part
    if index not in ind:
        ind.append(index)
    else:
       index = index + '1'
       ind.append(index)

100%|██████████| 15569/15569 [00:03<00:00, 3909.94it/s]


In [ ]:
len(ind), films.shape

(15569, (15569, 11))

In [ ]:
films['id'] = ind

In [ ]:
films.to_csv('movies.csv', index=False)

In [195]:
df = pd.read_csv('movies.csv')
df.head(1)

,Title,Type,Year,Age Limit,Episodes,Time,IMDB,Metascore,Overview,Box Office,Images,id
0,Conversations with Friends,TV Series,(2022),TV-MA,12.0,335 min,6.7,NaN,"Two Dublin college students, Frances and Bobbi...",NaN,conversations_with_friends_2022.jpg,cnvrstnswthfrnds36941


In [185]:
dt = pd.read_csv('movies_pr.csv')

In [209]:
df.Year[4237] = '(2010) (1)'
df.Year[7810] = '(2021) (1)'
df.Year[7941] = '(2018) (1)'
df.Year[9930] = '(1989–1996) (1)'
df.Year[11505] = '(1976) (1)'
df.Year[11605] = '(2005) (1)'
df.Year[11745] = '(2019) (1)'
df.Year[12220] = '(1980) (1)'
df.Year[12521] = '(1990) (1)'
df.Year[13606] = '(2022) (1)'
df.Year[13650] = '(2020) (1)'
df.Year[14139] = '(2017) (1)'
df.Year[14150] = '(2020) (1)'
df.Year[14291] = '(2005) (1)'
df.Year[14315] = '(2019) (1)'
df.Year[14697] = '(2021) (1)'

In [211]:
df.drop(index=13309, inplace=True)

In [ ]:
df[11605:]

In [212]:
print(df[df.duplicated(['Title', 'Year'])].shape)
df[df.duplicated(['Title', 'Year'])]

(0, 12)


,Title,Type,Year,Age Limit,Episodes,Time,IMDB,Metascore,Overview,Box Office,Images,id


In [ ]:
dt.to_csv('movies_pr.csv', index=False)

In [213]:
df.to_csv('movies.csv', index=False)

##stars

In [ ]:
dt = pd.read_csv('Prmovies10.csv')

In [ ]:
st =[]
for i in tqdm(range(len(dt))):
    if str(dt['Cast'][i]) != 'nan':
        st = st + str(dt['Cast'][i]).split(', ')
    if  str(dt['Directed'][i]) !='nan':
        st = st + str(dt['Directed'][i]).split(', ')

100%|██████████| 15569/15569 [00:48<00:00, 318.70it/s]


In [ ]:
s = pd.read_csv('Stars.csv')
s.head(1)

,id,name
0,evtmnzchcht86140,Evita Muñoz 'Chachita'


In [ ]:
stars = list(set(st))
# stars.remove('nan')
stars.sort(key=lambda x: x.split()[-1])
len(stars)

132961

In [ ]:
star = pd.DataFrame({'id':[0]*len(stars), 'name':stars})
star.head(2)

,id,name
0,0,$haniqua
1,0,Ty Dolla $ign


In [ ]:
for i in tqdm(range(len(star))):
    if star.name[i] in s.name.values:
        ind = s[s.name == star.name[i]].index[0]
        star.id[i] = s.id[ind]

100%|██████████| 27813/27813 [04:33<00:00, 101.87it/s]


In [ ]:
n = [str(i) for i in range(10)]
ind = []
flag = False
for i in tqdm(range(len(star))):
    line = str(star.name[i])
    line = re.sub(r"[&?.,\- \'!\$\(\) ’]", '', line)
    line = re.sub(r"[ÀÁÂÄÅÆ]", 'A', line).replace('Ç', 'C').replace('Þ', 'Th')
    line = re.sub(r"[ÈÉÊ]", 'E', line).replace('Í', 'I').replace('ß', 'Ss')
    line = re.sub(r"[ÒÓÔÕÖØ]", 'O', line).replace('Î', 'I')
    line = re.sub(r"[ÚÜ]", 'U', line).replace('Ý', 'Y')
    l_part = re.sub(r'(a|à|á|â|ã|ä|å|æ|e|è|é|ê|ë|i|ì|í|î|ï|o|ò|ó|ô|õ|ö|ő|ø|u|ù|ú|ū|û|ü|y|ý|ÿ)', '', line).lower()
    l_part = l_part.replace('þ', 'th').replace('ñ', 'n').replace('ð', 'd').replace('ß', 'ss').replace('ç', 'c')
    d_part = ''.join(np.random.choice(n, 5))
    index = l_part.lower() + d_part
    if index not in ind and index not in star.id.values:
        ind.append(index)
    else:
        index = index + '1'
        ind.append(index)

100%|██████████| 132961/132961 [03:24<00:00, 649.76it/s]


In [ ]:
len(ind), star.shape

(132961, (132961, 2))

In [ ]:
star.id = ind

In [ ]:
start, step  = 112210, 5
star[start:start+step]

,id,name
112210,mrjskngl28957,Marija Skangale
112211,nhskp74859,Noah Skape
112212,eldrskr82140,Eldar Skar
112213,tnskrbrg88260,Tony Skarberg
112214,ptrskrk44281,Petr Skarke


In [ ]:
star.to_csv('stars.csv', index=False)

## movies-genres

In [ ]:
k, ind = 0, []
for i in range(len(dt)):
    p = dt['Genre(s)'][i].split(',')
    k += len(p)
    for j in p:
        if j.strip() not in ind:
            ind.append(j.strip())
ind = sorted(ind)

In [ ]:
g = pd.DataFrame({'id':len(ind), 'genre':ind})

In [ ]:
n = [str(i) for i in range(10)]
ind = []
flag = False
for i in tqdm(range(len(g))):
    line = str(g.genre[i])
    line = re.sub(r"[aouiye\-]", '', line).lower()
    d_part = ''.join(np.random.choice(n, 2))
    g.id[i] = line + d_part

100%|██████████| 28/28 [00:00<00:00, 3204.82it/s]


In [ ]:
g.to_csv('genres.csv', index=False)

In [ ]:
mov_gen = pd.DataFrame({'movie_id':[0]*k, 'genre_id':[0]*k})

In [ ]:
gen, mov = [], []
for i in tqdm(range(len(dt))):
    jh = dt['Genre(s)'][i].split(', ')
    for j in range(len(jh)):
        x = g.id[g.genre == jh[j]].values[0]
        gen.append(x)
        mov.append(movies.id[i])

100%|██████████| 15569/15569 [00:08<00:00, 1832.21it/s]


In [ ]:
mov_gen['movie_id'] = mov
mov_gen['genre_id'] = gen
mov_gen.head(5)

,movie_id,genre_id
0,cnvrstnswthfrnds36941,drm55
1,cnvrstnswthfrnds36941,rmnc56
2,lstctn96373,drm55
3,lstctn96373,hstr95
4,lstctn96373,rmnc56


In [ ]:
mov_gen.to_csv('movies-genres.csv', index=False)

##movies-actors

In [ ]:
dt = pd.read_csv('movies_pr.csv')
movies = pd.read_csv('movies.csv')
star = pd.read_csv('movie-stars.csv')
mov_act = pd.read_csv('movie-actor.csv')

In [ ]:
fwna = dt[['Title', 'Cast']][dt.Cast.isna()==False]
fwna['id'] = movies['id']
fwna.reset_index(drop=True, inplace=True)
fwna.tail(5)

,Title,Cast,id
15550,Saekano: How to Raise a Boring Girlfriend,"Yoshitsugu Matsuoka, Saori Oonishi, Kiyono Yas...",sknhwtrsbrnggrlfrnd50105
15551,Bunnicula,"Chris Kattan, Sean Astin, Brian Kimmet, Kari W...",bnncl11327
15552,Maria Clara and Ibarra,"Barbie Forteza, Julie Anne San Jose, Dennis Tr...",mrclrndibrr65351
15553,Lore,"Aaron Mahnke, Paula Malcomson, Maimie McCoy, S...",lr19291
15554,Hazbin Hotel,"Amir Hossein Amirpour, Jill Harris, Elsie Love...",hzbnhtl42242


In [ ]:
k = 0
for i in tqdm(range(len(fwna))):
    k += len(fwna['Cast'][i].split(','))
k

100%|██████████| 15555/15555 [00:00<00:00, 34444.56it/s]


269039

In [ ]:
act, mov, ind = [], [], []
for i in tqdm(range(len(fwna))):
    jh = fwna['Cast'][i].split(', ')
    act = act + jh
    mov = mov + [fwna.id[i]]*len(jh)
    ind = ind + [j for j in range(1, len(jh) + 1)]

100%|██████████| 15555/15555 [00:58<00:00, 266.16it/s]


In [ ]:
m_a = pd.DataFrame({'movie_id':mov, 'actor_id':act, 'rating': ind})
print(m_a.shape)
m_a.drop_duplicates(['movie_id', 'actor_id'], keep='first', inplace=True)
m_a.reset_index(drop=True, inplace=True)
m_a.shape

(269039, 3)


(269010, 3)

In [ ]:
mov_act['rating'] = m_a['rating']

In [ ]:
for i in tqdm(range(len(mov_act))):
    mov_act.actor_id[i] = star.id[star.name==mov_act.actor_id[i]].values[0]

100%|██████████| 269039/269039 [1:06:23<00:00, 67.53it/s]


In [ ]:
m_a[m_a.actor_id=='Masako Watanabe']

,movie_id,actor_id
76716,onlystrd38865,Masako Watanabe
76717,onlystrd38865,Masako Watanabe


In [ ]:
mov_act[-5:], m_a[-5:]

(            movie_id           actor_id
 269034  hzbnhtl42242       mchllmr60852
 269035  hzbnhtl42242     dndrrlrvr99029
 269036  hzbnhtl42242  jmsmnriglhrt15458
 269037  hzbnhtl42242     mxwllatms97299
 269038  hzbnhtl42242     amnelgrch02333,
             movie_id               actor_id
 269034  hzbnhtl42242         Michelle Marie
 269035  hzbnhtl42242      Don Darryl Rivera
 269036  hzbnhtl42242  James Monroe Iglehart
 269037  hzbnhtl42242          Maxwell Atoms
 269038  hzbnhtl42242           Amin Elgarch)

In [ ]:
print(mov_act.shape)
mov_act.drop_duplicates(inplace=True, )
mov_act.shape

(269039, 2)


(269010, 2)

In [ ]:
st = 269006
m_a[st:st+5], mov_act[st:st+5]

(            movie_id               actor_id  rating
 269006  hzbnhtl42242      Don Darryl Rivera      15
 269007  hzbnhtl42242  James Monroe Iglehart      16
 269008  hzbnhtl42242          Maxwell Atoms      17
 269009  hzbnhtl42242           Amin Elgarch      18,
             movie_id           actor_id  rating
 269006  hzbnhtl42242     dndrrlrvr99029      15
 269007  hzbnhtl42242  jmsmnriglhrt15458      16
 269008  hzbnhtl42242     mxwllatms97299      17
 269009  hzbnhtl42242     amnelgrch02333      18)

In [ ]:
mov_act.to_csv('movie-actor.csv', index=False)

##movie-director

In [ ]:
fdwna = dt[['Title', 'Directed by']][dt['Directed by'].isna()==False]
fdwna['id'] = movies['id']
fdwna.reset_index(drop=True, inplace=True)
fdwna.tail(5)

,Title,Directed by,id
13799,Little Einsteins,Douglas Wood,lttlenstns61875
13800,Alpha Betas,"Chris Bruno, David Howard Lee",alphbts55689
13801,Transformers: Rescue Bots,"Nicole Dubuc, Brian Hohlfeld, Jeff Kline",trnsfrmrsrscbts91249
13802,Maria Clara and Ibarra,"Suzette Doctolero, Annette Gozon, RJ Nuevas",mrclrndibrr65351
13803,Hazbin Hotel,Vivienne Medrano,hzbnhtl42242


In [ ]:
k, c = 0, 0
for i in range(len(fdwna)):
    k += len(fdwna['Directed by'][i].split(','))
    if len(fdwna['Directed by'][i].split(',')) > c:
        c = len(fdwna['Directed by'][i].split(','))
        print(f'{i}) {fdwna.Title[i]} | {len(fdwna["Directed by"][i].split(", "))} | {fdwna["Directed by"][i]}')
k

0) Lust, Caution | 1 | Ang Lee
8) Ridley | 2 | Jonathan Fisher, Paul Matthew Thompson
127) Manhunt | 3 | Andrew Sodroski, Jim Clemente, Tony Gittelson


16143

In [ ]:
mov_dir = pd.DataFrame({'movie_id':[0]*k, 'director_id':[0]*k, 'rating':[0]*k})

In [ ]:
dir, m, r = [], [], []
for i in tqdm(range(len(fdwna))):
    jh = fdwna['Directed by'][i].split(', ')
    for j in range(len(jh)):
        x = star.id[star.name == jh[j]].values[0]
        dir.append(x)
        m.append(fdwna.id[i])
    r = r + [y for y in range(1, len(jh) + 1)]
len(m), len(dir), len(r)

100%|██████████| 13804/13804 [02:56<00:00, 78.19it/s]


(16143, 16143, 16143)

In [ ]:
mov_dir['movie_id'] = m
mov_dir['director_id'] = dir
mov_dir['rating'] = r
mov_dir.tail(5)

,movie_id,director_id,rating
16138,trnsfrmrsrscbts91249,jffkln34357,3
16139,mrclrndibrr65351,szttdctlr26861,1
16140,mrclrndibrr65351,annttgzn28283,2
16141,mrclrndibrr65351,rjnvs68196,3
16142,hzbnhtl42242,vvnnmdrn28739,1


In [ ]:
print(mov_dir.shape)
mov_dir.drop_duplicates(['movie_id', 'director_id'], inplace=True)
mov_dir.shape

(16143, 3)


(16140, 3)

In [ ]:
mov_dir.tail(15)

,movie_id,director_id,rating
16128,dngrms54765,andrwdlln73784,1
16129,rndcnnnghm9thgrdnnj54381,jdelnff66077,1
16130,rndcnnnghm9thgrdnnj54381,scttthms13619,2
16131,dcktwn53811,jhnhdgmn92787,1
16132,dcktwn53811,dvdrs30570,2
16133,lttlenstns61875,dglswd82212,1
16134,alphbts55689,chrsbrn02457,1
16135,alphbts55689,dvdhwrdl16273,2
16136,trnsfrmrsrscbts91249,ncldbc00961,1
16137,trnsfrmrsrscbts91249,brnhhlfld58456,2


In [ ]:
fdwna.tail(10)

,Title,Directed by,id
13794,Adventures of Sonic the Hedgehog,"Kent Butterworth, Phil Harnage, Bruce Shelly",advntrsfsncthhdghg49448
13795,Battle for Dream Island,"Cary Huang, Michael Huang, Tom McGillis",bttlfrdrmislnd82995
13796,Danger Mouse,Andrew Dillon,dngrms54765
13797,Randy Cunningham: 9th Grade Ninja,"Jed Elinoff, Scott Thomas",rndcnnnghm9thgrdnnj54381
13798,Dicktown,"John Hodgman, David Rees",dcktwn53811
13799,Little Einsteins,Douglas Wood,lttlenstns61875
13800,Alpha Betas,"Chris Bruno, David Howard Lee",alphbts55689
13801,Transformers: Rescue Bots,"Nicole Dubuc, Brian Hohlfeld, Jeff Kline",trnsfrmrsrscbts91249
13802,Maria Clara and Ibarra,"Suzette Doctolero, Annette Gozon, RJ Nuevas",mrclrndibrr65351
13803,Hazbin Hotel,Vivienne Medrano,hzbnhtl42242


In [ ]:
mov_dir.to_csv('movies-directors.csv', index=False)

#database

In [214]:
fm = pd.read_csv('movies.csv')
s = pd.read_csv('movie-stars.csv')
m_d = pd.read_csv('movies-directors.csv')
m_a = pd.read_csv('movie-actor.csv')
f_g = pd.read_csv('movies-genres.csv')
g = pd.read_csv('genres.csv')

In [215]:
print(f'Movie: {fm.head(1)}\n Stars: {s.head(1)}\n Genres: {g.head(1)}\n Mov-gen{f_g.head(1)}\n Mov-div{m_d.head(1)}\n Mov-act{m_a.head(1)}')

Movie:                         Title       Type    Year Age Limit  Episodes     Time  \
0  Conversations with Friends  TV Series  (2022)     TV-MA      12.0  335 min   

   IMDB  Metascore                                           Overview  \
0   6.7        NaN  Two Dublin college students, Frances and Bobbi...   

  Box Office                               Images                     id  
0        NaN  conversations_with_friends_2022.jpg  cnvrstnswthfrnds36941  
 Stars:          id      name
0  hnq28151  $haniqua
 Genres:        id   genre
0  actn25  Action
 Mov-gen                movie_id genre_id
0  cnvrstnswthfrnds36941    drm55
 Mov-div      movie_id director_id  rating
0  lstctn96373   angl79006       1
 Mov-act                movie_id       actor_id  rating
0  cnvrstnswthfrnds36941  alsnolvr43356       1


In [ ]:
conn = sqlite3.connect('Movies.db')
conn.execute("PRAGMA foreign_keys = 1")
conn.commit()

In [ ]:
curr = conn.cursor()
curr.execute("CREATE TABLE IF NOT EXISTS movies(\
    id TEXT PRIMARY KEY NOT NULL,\
    title TEXT NOT NULL, mtype TEXT, year TEXT,\
    age_limit TEXT, episodes INT, time TEXT, imdb REAL,\
    metascore INT, overview TEXT NOT NULL,\
    box_office TEXT, images TEXT NOT NULL)")
conn.commit()

In [ ]:
curr.execute("CREATE TABLE IF NOT EXISTS mov_gen( \
              movie_id TEXT NOT NULL,\
              genre_id TEXT NOT NULL,\
              PRIMARY KEY (movie_id, genre_id))")
curr.execute("CREATE TABLE IF NOT EXISTS genres( \
              id TEXT PRIMARY KEY NOT NULL,\
              genre TEXT NOT NULL)")
curr.execute("CREATE TABLE IF NOT EXISTS mov_dir( \
              movie_id TEXT NOT NULL,\
              dir_id TEXT NOT NULL, rating INT NOT NULL,\
              PRIMARY KEY (movie_id, dir_id))")
curr.execute("CREATE TABLE IF NOT EXISTS stars( \
              id TEXT PRIMARY KEY NOT NULL,\
              name TEXT NOT NULL)")
curr.execute("CREATE TABLE IF NOT EXISTS mov_act( \
              movie_id TEXT NOT NULL,\
              act_id TEXT NOT NULL, rating INT NOT NULL,\
              PRIMARY KEY (movie_id, act_id))")
conn.commit()

In [ ]:
curr.execute("SELECT name FROM sqlite_master WHERE type='table';")
curr.fetchall()

[('movies',),
 ('mov_gen',),
 ('genres',),
 ('mov_dir',),
 ('stars',),
 ('mov_act',)]

##stars

In [ ]:
conn = sqlite3.connect('Movies.db')
curr = conn.cursor()
curr.execute('Delete from mov_dir')
x = curr.fetchall()
curr.close()
conn.commit()
x

[]

In [ ]:
conn = sqlite3.connect('Movies.db')
curr = conn.cursor()
curr.execute('Select count(*) from stars')
x = curr.fetchall()
curr.close()
x

[(132961,)]

In [ ]:
s.shape

(132961, 2)

In [ ]:
step=10000
l = [[step*k+j-step for k in range(2)] for j in range(1*step, 14*step+1, step)]
l[-1][1] = s.shape[0]
l

[[0, 10000],
 [10000, 20000],
 [20000, 30000],
 [30000, 40000],
 [40000, 50000],
 [50000, 60000],
 [60000, 70000],
 [70000, 80000],
 [80000, 90000],
 [90000, 100000],
 [100000, 110000],
 [110000, 120000],
 [120000, 130000],
 [130000, 132961]]

In [ ]:
# Stars table
def f(a):
    start, stop = a
    conn = sqlite3.connect('Movies.db')
    for j in tqdm(range(start, stop), desc="Progress" ):
        curr = conn.cursor()
        curr.execute("INSERT INTO stars VALUES(?, ?);", \
                (s.id[j], s['name'][j]))
        curr.close()
    conn.commit()
try:
    print(Pool(1).map(f, l))
except (OperationalError, IntegrityError):
    pass
display(Audio('1.wav', autoplay=True))

Progress: 100%|██████████| 2961/2961 [00:00<00:00, 4700.21it/s]


[None, None, None, None, None, None, None, None, None, None, None, None, None, None]


## movies

In [ ]:
fm.columns.tolist()

['Title',
 'Type',
 'Year',
 'Age Limit',
 'Episodes',
 'Time',
 'IMDB',
 'Metascore',
 'Overview',
 'Box Office',
 'Images',
 'id']

In [216]:
fm.shape

(15568, 12)

In [217]:
step=10000
l = [[step*k+j-step for k in range(2)] for j in range(1*step, 2*step+1, step)]
l[-1][1] = fm.shape[0]
l

[[0, 10000], [10000, 15568]]

In [218]:
conn = sqlite3.connect('Movies.db')
curr = conn.cursor()
curr.execute('Delete from movies')
x = curr.fetchall()
curr.close()
conn.commit()
x

[]

In [219]:
conn = sqlite3.connect('Movies.db')
curr = conn.cursor()
curr.execute('Select * from movies Limit 5')
x = curr.fetchall()
curr.close()
x

[]

In [220]:
# Films table
def f(a):
    start, stop = a
    conn = sqlite3.connect('Movies.db')
    for i in tqdm(range(start, stop), desc="Progress"):
        curr = conn.cursor()
        try:
            epi = int(fm.Episodes[i])
        except ValueError:
            epi = fm.Episodes[i]
        try:
            meta = int(fm.Metascore[i])
        except ValueError:
            meta = fm.Metascore[i]
        curr.execute("INSERT INTO movies VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", \
                (fm.id[i], fm.Title[i], fm['Type'][i], fm.Year[i], fm['Age Limit'][i],\
                epi, fm.Time[i], fm.IMDB[i], meta, fm['Overview'][i],\
                fm['Box Office'][i], fm.Images[i], ))
        curr.close()
    conn.commit()
try:
    print(Pool(1).map(f, l))
except (OperationalError, IntegrityError):
    pass
display(Audio('1.wav', autoplay=True))

Progress: 100%|██████████| 5568/5568 [00:00<00:00, 7970.13it/s]


[None, None]


##genres


In [ ]:
g.columns.tolist()

['id', 'genre']

In [ ]:
# Genre table
conn = sqlite3.connect('Movies.db')
curr = conn.cursor()
for j in tqdm(range(len(g)), desc="Progress" ):
  curr.execute("INSERT INTO genres VALUES(?, ?);", \
              (g.id[j], g.genre[j]))
  conn.commit()
curr.close()

Progress: 100%|██████████| 28/28 [00:00<00:00, 52.04it/s]


##mov_gen

In [ ]:
f_g.columns.tolist()

['movie_id', 'genre_id']

In [ ]:
f_g.shape

(41437, 2)

In [ ]:
step=10000
l = [[step*k+j-step for k in range(2)] for j in range(1*step, 5*step+1, step)]
l[-1][1] = f_g.shape[0]
l

[[0, 10000], [10000, 20000], [20000, 30000], [30000, 40000], [40000, 41437]]

In [ ]:
# Genre_ids table
def f(a):
    start, stop = a
    conn = sqlite3.connect('Movies.db')
    for j in tqdm(range(start, stop), desc="Progress" ):
        curr = conn.cursor()
        curr.execute("INSERT INTO mov_gen VALUES(?, ?);", \
              (f_g.movie_id[j], f_g.genre_id[j]))
        curr.close()
    conn.commit()
try:
    print(Pool(1).map(f, l))
except (OperationalError, IntegrityError):
    pass
display(Audio('1.wav', autoplay=True))


Progress: 100%|██████████| 1437/1437 [00:00<00:00, 7546.58it/s]


[None, None, None, None, None]


##mov_act

In [ ]:
conn = sqlite3.connect('Movies.db')
curr = conn.cursor()
curr.execute('Delete from mov_act')
x = curr.fetchall()
curr.close()
conn.commit()
x

[]

In [ ]:
conn = sqlite3.connect('Movies.db')
curr = conn.cursor()
curr.execute('Select * from mov_act limit 5')
x = curr.fetchall()
curr.close()
x

[('cnvrstnswthfrnds36941', 'alsnolvr43356', 1),
 ('cnvrstnswthfrnds36941', 'sshln77088', 2),
 ('cnvrstnswthfrnds36941', 'jalwn38542', 3),
 ('cnvrstnswthfrnds36941', 'jmmkrk06545', 4),
 ('cnvrstnswthfrnds36941', 'alxmrph92475', 5)]

In [ ]:
m_a.shape

(269010, 2)

In [ ]:
m_a.columns.tolist()

['movie_id', 'actor_id', 'rating']

In [ ]:
step=10000
l = [[step*k+j-step for k in range(2)] for j in range(1*step, 27*step+1, step)]
l[-1][1] = m_a.shape[0]
l

In [ ]:
def f(a):
    start, stop = a
    conn = sqlite3.connect('Movies.db')
    for j in tqdm(range(start, stop), desc="Progress" ):
        curr = conn.cursor()
        curr.execute("INSERT INTO mov_act VALUES(?, ?, ?);", \
                (m_a.movie_id[j], m_a.actor_id[j], int(m_a.rating[j])))
        curr.close()
    conn.commit()
try:
    print(Pool(1).map(f, l))
except (OperationalError):
    pass
display(Audio('1.wav', autoplay=True))

##mov_dir

In [ ]:
conn = sqlite3.connect('Movies.db')
curr = conn.cursor()
curr.execute('Delete from mov_dir')
x = curr.fetchall()
curr.close()
conn.commit()
x

[]

In [ ]:
conn = sqlite3.connect('Movies.db')
curr = conn.cursor()
curr.execute('Select count(*) from mov_dir')
x = curr.fetchall()
curr.close()
x

[(16140,)]

In [ ]:
m_d.shape

(16140, 2)

In [ ]:
m_d.columns.tolist()

['movie_id', 'director_id']

In [ ]:
step = 10000
l = [[step*k + j - step for k in range(2)] for j in range(1*step, 2*step + 1, step)]
l[-1][1] = m_d.shape[0]
l

[[0, 10000], [10000, 16140]]

In [ ]:
# Directors table
def f(a):
    start, stop = a
    conn = sqlite3.connect('Movies.db')
    for j in tqdm(range(start, stop), desc="Progress"):
        curr = conn.cursor()
        curr.execute("INSERT INTO mov_dir VALUES(?, ?, ?);", \
                (m_d['movie_id'][j], m_d['director_id'][j], int(m_d['rating'][j])))
        curr.close()
    conn.commit()
try:
    print(Pool(1).map(f, l))
except (OperationalError, IntegrityError):
    pass
display(Audio('1.wav', autoplay=True))

Progress: 100%|██████████| 6140/6140 [00:00<00:00, 7706.34it/s]


[None, None]


##queries

In [ ]:
sql = '''
Select *
from movies as m
inner join mov_dir as md on m.id = md.movie_id
inner join stars as s on ma.act_id = s.id
inner join mov_act as ma on m.id = ma.movie_id
inner join mov_gen as mg on m.id = mg.movie_id
inner join genres as g on mg.movie_id = m.id
Order by name, title, year, rating
limit 2
'''
conn = sqlite3.connect('Movies.db')
curr = conn.cursor()
curr.execute(sql)
x = curr.fetchall()
curr.close()
conn.commit()
for i in x:
    print(i)

('bxcrbrth23329', 'Boxcar Bertha', 'Movie', '(1972)', 'R', None, '88 min', 6.0, 61, 'During the Great Depression, a union leader and a young woman become criminals to exact revenge on the management of a railroad.', None, 'boxcar_bertha_1972.jpg', 'bxcrbrth23329', 'mrtnscrss13008', 1, 'chcknhllmn83530', "'Chicken' Holleman", 'bxcrbrth23329', 'chcknhllmn83530', 9, 'bxcrbrth23329', 'crm11', 'actn25', 'Action')
('bxcrbrth23329', 'Boxcar Bertha', 'Movie', '(1972)', 'R', None, '88 min', 6.0, 61, 'During the Great Depression, a union leader and a young woman become criminals to exact revenge on the management of a railroad.', None, 'boxcar_bertha_1972.jpg', 'bxcrbrth23329', 'mrtnscrss13008', 1, 'chcknhllmn83530', "'Chicken' Holleman", 'bxcrbrth23329', 'chcknhllmn83530', 9, 'bxcrbrth23329', 'crm11', 'adlt81', 'Adult')


In [221]:
sql = '''
Select title, year, imdb, metascore, rating,  name
from movies as m
inner join mov_act as ma on m.id = ma.movie_id
inner join stars as s on ma.act_id = s.id
where name like "%Selena %"
Order by name, title, year, rating
'''
conn = sqlite3.connect('Movies.db')
curr = conn.cursor()
curr.execute(sql)
x = curr.fetchall()
curr.close()
conn.commit()
for i in x:
    print(i)

('Doctor Sleep', '(2019)', 7.3, 59, 7, 'Selena Anduze')
('Swamp Thing', '(2019)', 7.4, None, 11, 'Selena Anduze')
('WandaVision', '(2021)', 7.9, None, 9, 'Selena Anduze')
('A Rainy Day in New York', '(2019)', 6.5, 38, 8, 'Selena Gomez')
('Arthur et la vengeance de Maltazard', '(2009)', 5.2, None, 1, 'Selena Gomez')
('Behaving Badly', '(2014)', 4.4, 18, 2, 'Selena Gomez')
('Dolittle', '(2020)', 5.6, 26, 15, 'Selena Gomez')
('Hotel Transylvania', '(2012)', 7.0, 47, 4, 'Selena Gomez')
('Hotel Transylvania 2', '(2015)', 6.6, 44, 3, 'Selena Gomez')
('Hotel Transylvania 3: Summer Vacation', '(2018)', 6.3, 54, 3, 'Selena Gomez')
('Hotel Transylvania: Transformania', '(2022)', 6.0, 46, 2, 'Selena Gomez')
('In Dubious Battle', '(2016)', 6.0, 43, 4, 'Selena Gomez')
('Monte Carlo', '(2011)', 5.8, 43, 1, 'Selena Gomez')
('Neighbors 2: Sorority Rising', '(2016)', 5.7, 58, 13, 'Selena Gomez')
('Only Murders in the Building', '(2021– )', 8.1, None, 3, 'Selena Gomez')
('Princess Protection Program', '